In [29]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.llms import OpenAI
import io
import requests
from pypdf import PdfReader
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [30]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

url = 'https://www.towerhamlets.gov.uk/Documents/Planning-and-building-control/Strategic-Planning/Local-Plan/TH_Local_Plan_2031_accessibility_checked.pdf'
response = requests.get(url=url, headers=headers, timeout=120)
on_fly_mem_obj = io.BytesIO(response.content)
reader = PdfReader(on_fly_mem_obj)

output = ""
for i in range(len(reader.pages)):
    page = reader.pages[i]
    output += page.extract_text()

print(output[0:200])

Tower Hamlets Plan 2031   Managing Growth and Sharing Benefits1Tower Hamlets 
Local Plan 2031
Adopted January 2020
Managing growth and
sharing the beneﬁts
Produced by:
Strategic Planning T eam  
Plann


In [31]:

davinci = OpenAI(model_name='text-embedding-ada-002')

In [23]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)

text = text_splitter.create_documents([output])

splits = text_splitter.split_documents(text)


db.add_documents(splits)

['4236dde8-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e004-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e022-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e02c-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e040-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e054-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e05e-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e072-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e086-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e09a-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e0ae-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e0b8-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e0cc-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e0d6-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e0ea-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e0f4-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e108-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e112-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e126-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e130-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e144-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e14e-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e162-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e16c-92f2-11ee-9e4d-e6b84e5c62cd',
 '4236e180-92f2-

In [19]:
retriever = db.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
rag_chain.invoke("How will Tower Hamlets acheive sustainable growth?")

'Tower Hamlets will achieve sustainable growth by delivering managed growth through good design, preserving or enhancing the character and setting of the area, and avoiding negative impacts on the environment and infrastructure. They will also share the benefits of growth by creating healthy environments, promoting mixed and balanced communities, and delivering tenure-blind developments. Additionally, they will provide a range of housing typologies, maximize provision of amenity space, and support the provision of family housing and affordable workspace.'